In [ ]:
# Check if python is 3.9.5
import arrow
import numpy as np
import os
import sys

from __init__ import PCAT

print(sys.version)
%load_ext autoreload
%autoreload 2
%pprint

# How much input can we get from other CliMA models?

## Macroecology of CliMA models

Actually, the [CliMA](https://clima.caltech.edu/) models are designed in a way that allows to connect them all. There is a [blog entry](https://clima.caltech.edu/2022/02/09/toward-a-new-earth-system-model-clima-at-agu/) about it, but [this poster](https://clima.caltech.edu/files/2022/02/MackayAGU.pdf) explains it best:

<details style="background-color:#eeeeee">
    <summary>Show poster:</summary>
    <iframe src="https://clima.caltech.edu/files/2022/02/MackayAGU.pdf" style="width:50%">
    </iframe>
</details>

In short, [`ClimaCoupler.jl`](https://github.com/CliMA/ClimaCoupler.jl) is designed to do this mediation between different models. 

Another design choice of CliMA models is [`ClimaCore.jl`](https://github.com/CliMA/ClimaCore.jl), an interface that allows to felixible space discretizations and thus delivers a base for all models. [Here](https://agu2021fallmeeting-agu.ipostersessions.com/default.aspx?s=43-72-C2-E4-F0-E6-33-EE-F7-EC-3E-83-8D-5C-E8-63&guestview=true) is some more explanation:

<details style="background-color:#eeeeee">
    <summary>Show poster:</summary>
    <iframe src="https://agu2021fallmeeting-agu.ipostersessions.com/default.aspx?s=43-72-C2-E4-F0-E6-33-EE-F7-EC-3E-83-8D-5C-E8-63&guestview=true" style="width:50%">
    </iframe>
</details>

To understand `ClimaCore.jl` a bit better, I did the [tutorial](https://github.com/CliMA/ClimaWorkshops) which I stored in the [`ext_src` directory](../../../ext_src/02_tutorials/01_climacore/ClimaWorkshops/README.md).